In [1]:
import os
import sys

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(PROJECT_ROOT)

from trial_class import *
from experiment_class import Experiment

import numpy as np
import pandas as pd
import tdt
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from scipy.signal import butter, filtfilt
from sklearn.linear_model import LinearRegression
from sp_extension import *

In [7]:
experiment_path = r"/Users/naylajimenez/Downloads/papers/dopamine/cohort-1-2/allcohorts/C1_2_3_Social_Pref/all_c3_nac"
csv_base_path = r"/Users/naylajimenez/Downloads/papers/dopamine/cohort-1-2/allcohorts/C1_2_3_Social_Pref/all_csvs"

# NAc: #15616F
# mPFC: #FFAF00

In [8]:
# groups csv + experiment data into one variable
experiment = Experiment(experiment_path, csv_base_path)

# batch process the data, removing the specified time segments for subjects
experiment.default_batch_process()

Found Synapse note file: /Users/naylajimenez/Downloads/papers/dopamine/cohort-1-2/allcohorts/C1_2_3_Social_Pref/all_c3_nac/p5-240826-091418/Notes.txt
read from t=0s to t=790.95s
Found Synapse note file: /Users/naylajimenez/Downloads/papers/dopamine/cohort-1-2/allcohorts/C1_2_3_Social_Pref/all_c3_nac/nn2-250117-085631/Notes.txt
read from t=0s to t=814.05s
Found Synapse note file: /Users/naylajimenez/Downloads/papers/dopamine/cohort-1-2/allcohorts/C1_2_3_Social_Pref/all_c3_nac/p2-240523-081105/Notes.txt
read from t=0s to t=788.65s
Found Synapse note file: /Users/naylajimenez/Downloads/papers/dopamine/cohort-1-2/allcohorts/C1_2_3_Social_Pref/all_c3_nac/nn4-250118-094351/Notes.txt
read from t=0s to t=795.95s
Found Synapse note file: /Users/naylajimenez/Downloads/papers/dopamine/cohort-1-2/allcohorts/C1_2_3_Social_Pref/all_c3_nac/nn1-250117-081652/Notes.txt
read from t=0s to t=870.75s
Found Synapse note file: /Users/naylajimenez/Downloads/papers/dopamine/cohort-1-2/allcohorts/C1_2_3_Social_

In [10]:
bout_definitions = [
    {'prefix': 'Subject', 'introduced': 'Subject Introduced', 'removed': 'Subject Removed'},
]


experiment.group_extract_manual_annotations(bout_definitions,first_only=True)

Processing behaviors for p5-240826-091418...
Processing behaviors for nn2-250117-085631...
Processing behaviors for p2-240523-081105...
Processing behaviors for nn4-250118-094351...
Processing behaviors for nn1-250117-081652...
Processing behaviors for pp7-250118-075659...
Processing behaviors for pp3-250118-064713...
Processing behaviors for p8-240827-075823...
Processing behaviors for nn8-250118-105443...
Processing behaviors for p1-240522-080200...
Processing behaviors for p6-240827-065303...
Processing behaviors for pp5-250117-121543...
Processing behaviors for pp4-250118-072201...
Processing behaviors for nn7-250118-101917...
Processing behaviors for pp8-250118-083250...
Processing behaviors for nn6-250117-101903...
Processing behaviors for p7-240826-102402...
Processing behaviors for pp6-250117-124823...
Processing behaviors for p3-240522-092431...
Processing behaviors for nn3-250118-090940...
Processing behaviors for nn5-250117-093631...
Processing behaviors for p4-240523-092600

## Dopamine Stuff (Need help with this)

In [11]:
cups = "/Users/naylajimenez/Downloads/papers/dopamine/cohort-1-2/allcohorts/Social_Pref_Cup_Assignments.csv"

In [12]:
experiment.compute_all_da_metrics(use_max_length=False,
                                  max_bout_duration=5, #total_avg_bout_duration
                                  use_adaptive=False, 
                                  allow_bout_extension=False,
                                  mode='standard')

Computing DA metrics for p5-240826-091418 ...
Computing DA metrics for nn2-250117-085631 ...
Computing DA metrics for p2-240523-081105 ...
Computing DA metrics for nn4-250118-094351 ...
Computing DA metrics for nn1-250117-081652 ...
Computing DA metrics for pp7-250118-075659 ...
Computing DA metrics for pp3-250118-064713 ...
Computing DA metrics for p8-240827-075823 ...
Computing DA metrics for nn8-250118-105443 ...
Computing DA metrics for p1-240522-080200 ...
Computing DA metrics for p6-240827-065303 ...
Computing DA metrics for pp5-250117-121543 ...
Computing DA metrics for pp4-250118-072201 ...
Computing DA metrics for nn7-250118-101917 ...
Computing DA metrics for pp8-250118-083250 ...
Computing DA metrics for nn6-250117-101903 ...
Computing DA metrics for p7-240826-102402 ...
Computing DA metrics for pp6-250117-124823 ...
Computing DA metrics for p3-240522-092431 ...
Computing DA metrics for nn3-250118-090940 ...
Computing DA metrics for nn5-250117-093631 ...
Computing DA metrics

In [13]:
non_empty_trials = []
empty_trials = []

for trial_name, trial in experiment.trials.items():
    if hasattr(trial, "behaviors") and not trial.behaviors.empty:
        non_empty_trials.append(trial_name)
    else:
        empty_trials.append(trial_name)

print(f"\n✅ Trials with DA metrics: {len(non_empty_trials)} / {len(experiment.trials)}")
print(f"🧪 Trials with data: {non_empty_trials}")
print(f"⚠️  Trials with empty DA metrics: {empty_trials}")


✅ Trials with DA metrics: 8 / 24
🧪 Trials with data: ['p5-240826-091418', 'p2-240523-081105', 'p8-240827-075823', 'p1-240522-080200', 'p6-240827-065303', 'p7-240826-102402', 'p3-240522-092431', 'p4-240523-092600']
⚠️  Trials with empty DA metrics: ['nn2-250117-085631', 'nn4-250118-094351', 'nn1-250117-081652', 'pp7-250118-075659', 'pp3-250118-064713', 'nn8-250118-105443', 'pp5-250117-121543', 'pp4-250118-072201', 'nn7-250118-101917', 'pp8-250118-083250', 'nn6-250117-101903', 'pp6-250117-124823', 'nn3-250118-090940', 'nn5-250117-093631', 'pp1-250117-110456', 'pp2-250117-113909']


In [ ]:
def get_trial_dataframes(experiment):
    """
    Given an Experiment object, return a dictionary where:
    - Keys are subject IDs (Trial.subject_name).
    - Values are DataFrames corresponding to the behaviors of each trial.
    """
    trial_data = {}

    for trial in experiment.trials.values():
        subject_id = trial.subject_name  # Extract subject ID
        trial_data[subject_id] = trial.behaviors  # Store behaviors DataFrame

    return trial_data


exp_da_dict = get_trial_dataframes(experiment)

In [ ]:
def create_da_metrics_dataframe(trial_data, behavior="Investigation", desired_bouts=None):
    """
    Parameters
    ----------
    trial_data : dict
        Dictionary of { subject_id : DataFrame }, 
        where each DataFrame has columns: 
        [Bout, Behavior, AUC, Max Peak, Mean Z-score, etc.]

    behavior : str, optional
        The behavior type to filter for (default = "Investigation").

    desired_bouts : list or None, optional
        A list of bout labels to keep. If None, all bouts present in the subject's DataFrame are retained.

    Returns
    -------
    pd.DataFrame
        A DataFrame with columns:
        [Subject, Bout, Behavior, AUC, Max Peak, Mean Z-score].
        - If multiple events occur in the same bout, the metrics are averaged.
        - If the bout has no rows for the specified behavior, all metrics are set to 0.
    """
    metric_rows = []

    # Loop over each subject and its corresponding DataFrame
    for subject_id, df in trial_data.items():
        
        # Determine which bouts to include
        if desired_bouts is not None:
            bouts = desired_bouts
        else:
            bouts = df["Bout"].unique()

        # Process each bout for the current subject
        for bout in bouts:
            # Filter the subject's DataFrame for the current bout
            df_bout = df[df["Bout"] == bout]
            # Further filter by the specified behavior
            df_behavior = df_bout[df_bout["Behavior"] == behavior]
            
            if df_behavior.empty:
                # No events of this behavior => metrics are 0
                auc_val = 0
                max_peak_val = 0
                mean_z_val = 0
            else:
                # If there are multiple rows, we average these metrics
                auc_val = df_behavior["AUC"].mean()
                max_peak_val = df_behavior["Max Peak"].mean()
                mean_z_val = df_behavior["Mean Z-score"].mean()

            metric_rows.append({
                "Subject": subject_id,
                "Bout": bout,
                "Behavior": behavior,
                "AUC": auc_val,
                "Max Peak": max_peak_val,
                "Mean Z-score": mean_z_val
            })

    # Concatenate all rows into a single DataFrame
    final_df = pd.DataFrame(metric_rows)
    
    return final_df
desired_bouts=['sniff cup 1', 'sniff cup 2', 'sniff cup 3', 'sniff cup 4']

da_metadata_df = create_da_metrics_dataframe(exp_da_dict, behavior="Investigation", desired_bouts=desired_bouts)
da_metadata_df

In [ ]:
def dopamine(precomputed_df, 
             metric_name="Mean Z-score", 
             title="Combined DA Metrics", 
             ylabel="DA Metric", 
             xlabel="Bout", 
             custom_xtick_labels=None, 
             custom_xtick_colors=None, 
             ylim=None, 
             bar_color="#00B7D7", 
             yticks_increment=None, 
             figsize=(14,8), 
             pad_inches=0.1,
             save=False,
             save_name=None,
             subjects_to_include=None):  # ✅ New parameter
    """
    Plots DA metrics across specific bouts using a provided DataFrame.
    Optionally filters to only include specified subjects.
    Prints paired t-test results comparing first bout to others.
    """
    from scipy.stats import ttest_rel
    import numpy as np
    import matplotlib.pyplot as plt

    def perform_t_tests(pivot_df):
        results = {}
        bout_names = pivot_df.columns.tolist()
        for i in range(1, len(bout_names)):
            bout1, bout2 = bout_names[0], bout_names[i]
            paired_df = pivot_df[[bout1, bout2]].dropna()
            if len(paired_df) > 1:
                t_stat, p_value = ttest_rel(paired_df[bout1], paired_df[bout2])
                significance = "ns"
                if p_value < 0.001:
                    significance = "***"
                elif p_value < 0.01:
                    significance = "**"
                elif p_value < 0.05:
                    significance = "*"
                results[f"{bout1} vs {bout2}"] = {
                    "t_stat": t_stat, 
                    "p_value": p_value, 
                    "significance": significance
                }
        return results

    df = precomputed_df.copy()

    # ✅ Filter to specified subjects
    if subjects_to_include is not None:
        df = df[df["Subject"].isin(subjects_to_include)]
        if df.empty:
            print("No data remaining after subject filtering.")
            return

    # Pivot for plotting
    try:
        pivot_df = df.pivot(index="Subject", columns="Bout", values=metric_name)
    except Exception as e:
        print("Error pivoting data:", e)
        return

    overall_stats = df.groupby("Bout")[metric_name].agg(['mean', 'sem']).reset_index()
    t_test_results = perform_t_tests(pivot_df)

    fig, ax = plt.subplots(figsize=figsize)
    bar_positions = np.arange(len(overall_stats))

    ax.bar(bar_positions, overall_stats["mean"], yerr=overall_stats["sem"],
           capsize=6, color=bar_color, edgecolor='black', linewidth=5, width=0.6,
           error_kw=dict(elinewidth=3, capthick=3, zorder=5))

    for subject_id in pivot_df.index:
        ax.plot(bar_positions, pivot_df.loc[subject_id], linestyle='-', color='gray', 
                alpha=0.5, linewidth=3, marker='o', markerfacecolor='none', 
                markeredgecolor='gray', markeredgewidth=2, markersize=10)

    ax.set_ylabel(ylabel, fontsize=35, labelpad=12)
    ax.set_xlabel(xlabel, fontsize=35, labelpad=12)
    ax.set_title(title, fontsize=28)

    ax.set_xticks(bar_positions)
    xtick_labels = custom_xtick_labels if custom_xtick_labels else overall_stats["Bout"].tolist()
    ax.set_xticklabels(xtick_labels, fontsize=28)
    if custom_xtick_colors:
        for tick, color in zip(ax.get_xticklabels(), custom_xtick_colors):
            tick.set_color(color)

    ax.tick_params(axis='y', labelsize=35)
    ax.tick_params(axis='x', labelsize=35)

    if ylim:
        ax.set_ylim(ylim)
    else:
        all_vals = np.concatenate([pivot_df.values.flatten(), overall_stats["mean"].values])
        ax.set_ylim(0, np.nanmax(all_vals) * 1.2)

    if yticks_increment:
        y_min, y_max = ax.get_ylim()
        ax.set_yticks(np.arange(np.floor(y_min), np.ceil(y_max) + yticks_increment, yticks_increment))

    ax.axhline(y=0, color='black', linestyle='--', linewidth=2)

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_linewidth(5)
    ax.spines['bottom'].set_linewidth(5)

    plt.tight_layout()
    if save:
        if save_name is None:
            raise ValueError("save_name must be provided if save is True.")
        plt.savefig(save_name, transparent=True, bbox_inches='tight', pad_inches=pad_inches)
    plt.show()

    if t_test_results:
        print("\nPaired t-test results (comparing first bout to others):")
        for comparison, stats in t_test_results.items():
            print(f"{comparison}: p = {stats['p_value']:.4f} ({stats['significance']})")

Pref = ['p5-240826-091418', 'pp3-250118-064713', 'pp5-250117-121543', 'pp4-250118-072201', 'p7-240826-102402', 'nn3-250118-090940', 'p4-240523-092600']
No_Pref = ['nn2-250117-085631', 'p2-240523-081105', 'nn4-250118-094351', 'nn1-250117-081652', 'pp7-250118-075659', 'p8-240827-075823', 'nn8-250118-105443', 'p1-240522-080200', 'p6-240827-065303', 'nn7-250118-101917', 'pp8-250118-083250', 'nn6-250117-101903', 'pp6-250117-124823', 'p3-240522-092431', 'nn5-250117-093631', 'pp1-250117-110456', 'pp2-250117-113909']

dopamine(
    precomputed_df=df,
    metric_name="Max Peak",
    title=None,
    ylabel="Peak Z-scored ∆F/F",
    xlabel="Bout Type",
    custom_xtick_labels=clean_labels,
    custom_xtick_colors=xtick_colors,
    ylim=(-1, 9),
    yticks_increment=2,
    bar_color='white',
    figsize=(14, 8),
    save=True,
    save_name ="mPFC_C1_3"
)

## Random Stuff